# MS 2.2- Group 10

**Members**: Niklas Grüner (12217059), Konstantin Unterweger (12222169), Martin Harhammer (12221683)


The following sections describe and implement an attempt to Audio Identification. 

## Imports

In [1]:
# imports
import os, sys
import numpy as np
from numba import jit
import librosa
#from scipy import signal
from scipy import ndimage
from matplotlib import pyplot as plt
import IPython.display as ipd
import time

sys.path.append('..')
import libfmp.b
import libfmp.c2
import libfmp.c6
from IPython.display import clear_output
from collections import defaultdict
import math
import matplotlib.pyplot as plt
import pickle 

import gc


%matplotlib inline

## Utility functions

Here we define all functions that, we later need to compute spectrograms, constellation maps, the hashes and perform the matching

In [2]:
def load_filenames(directory):
    filenames = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        # Create the full path
        file_path = os.path.join(directory, filename)
        
        # Check if it is a file (not a directory)
        if os.path.isfile(file_path):
            # Add to the dictionary, using the filename as the key
            filenames.append(file_path)

    return filenames

In [3]:
def compute_spectrogram(fn_wav, Fs=22050, N=2048, H=1024, bin_max=128, frame_max=None, duration=None):
    x, Fs = librosa.load(fn_wav, sr=Fs)
    x_duration = len(x) / Fs
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hann')
    if bin_max is None:
        bin_max = X.shape[0]
    if frame_max is None:
        frame_max = X.shape[0]
    Y = np.abs(X[:bin_max, :frame_max])
    return Y

In [4]:
def compute_constellation_map(Y, dist_freq=7, dist_time=7, thresh=0.01):
    """Compute constellation map (implementation using image processing)

    Notebook: C7/C7S1_AudioIdentification.ipynb

    Args:
        Y (np.ndarray): Spectrogram (magnitude)
        dist_freq (int): Neighborhood parameter for frequency direction (kappa) (Default value = 7)
        dist_time (int): Neighborhood parameter for time direction (tau) (Default value = 7)
        thresh (float): Threshold parameter for minimal peak magnitude (Default value = 0.01)

    Returns:
        Cmap (np.ndarray): Boolean mask for peak structure (same size as Y)
    """
    result = ndimage.maximum_filter(Y, size=[2*dist_freq+1, 2*dist_time+1], mode='constant')
    Cmap = np.logical_and(Y == result, result > thresh)
    return Cmap

In [5]:
from concurrent.futures import ThreadPoolExecutor

def compute_constellation_map_single(args):
    """Compute the constellation map for a single file."""
    filename, dist_freq, dist_time = args
    spectrogram = compute_spectrogram(filename)  # Perform I/O and computation
    constellation_map = compute_constellation_map(spectrogram, dist_freq, dist_time)
    return filename, constellation_map

def compute_constellation_maps(filenames, dist_freq, dist_time):
    """Compute constellation maps using multithreading."""
    # Prepare arguments for each file
    args = [(filename, dist_freq, dist_time) for filename in filenames]
    
    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor() as executor:
        results = executor.map(compute_constellation_map_single, args)
    
    # Convert results to a dictionary
    Cmaps = dict(results)
    return Cmaps

In [6]:
count = 0
count2 = 0

In [7]:

def compute_hashes_from_spectrogram(filename, dist_freq, dist_time, 
                                    time_min_offset, time_max_offset, 
                                    freq_min_offset, freq_max_offset):
    """
    Compute hashes for a single track directly from its spectrogram.
    
    Args:
        filename (str): Path to the audio file.
        dist_freq (int): Neighborhood parameter for frequency direction.
        dist_time (int): Neighborhood parameter for time direction.
        time_min_offset, time_max_offset, freq_min_offset, freq_max_offset (int): 
            Bounds for the "target zone" around an anchor point.

    Returns:
        list: [(hash, track_name, time_offset), ...]
    """

    global count
    global count2
    spectrogram = compute_spectrogram(filename)
    cmap = compute_constellation_map(spectrogram, dist_freq, dist_time)
    
    # Convert the boolean map to a list of points (freq, time), sorted by time
    point_list = sorted(np.argwhere(cmap).tolist(), key=lambda x: x[1])
    hashes = []
    del cmap
    del spectrogram
    
    for anchor in point_list:
        target_points = get_target_zone_points(
            anchor, 
            point_list,
            time_min_offset=time_min_offset,
            time_max_offset=time_max_offset,
            freq_min_offset=freq_min_offset,
            freq_max_offset=freq_max_offset
        )
        for target_point in target_points:
            h = compute_hash(anchor, target_point)
            hashes.append((h, int(extract_numeric_id(filename)), anchor[1]))
    
    count += 1
    count2 += 1
    if (count % 1000) == 0:
        gc.collect()
        print(f"Processed {count} tracks")

    return hashes




def build_database(
    directory,
    dist_freq=11,
    dist_time=3,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    Build a fingerprint database directly from spectrograms using multithreading.
    
    Args:
        directory (str): Directory containing audio files.
        dist_freq, dist_time (int): Constellation map neighborhood parameters.
        time_min_offset, time_max_offset, freq_min_offset, freq_max_offset (int): 
            Bounds for the "target zone" around an anchor point.

    Returns:
        database (defaultdict): {hash: [(track_name, time_offset), ...]}
    """
    tracks = load_filenames(directory)
    
    # Prepare arguments for each file
    args = [
        (filename, dist_freq, dist_time, time_min_offset, time_max_offset, freq_min_offset, freq_max_offset) 
        for filename in tracks
    ]
    
    database = defaultdict(list)
    global count2

    # Use multithreading to compute hashes for all tracks
    with ThreadPoolExecutor() as executor:
        for result in executor.map(lambda x: compute_hashes_from_spectrogram(*x), args):
            for h, track_name, time_offset in result:
                database[h].append((track_name, time_offset))

            if(count2 > 1000):
                count2 = 0
                save_partial_database(database, "partial_database2.pkl")
                database.clear()
    
    save_partial_database(database, "partial_database2.pkl")
    database.clear()
    return database





In [8]:
def compute_hash(anchor, target):
    """Generate a 32-bit hash."""
    f1, t1 = anchor
    f2, t2 = target
    dt = t2 - t1
    return (f1 & 0x3FF) | ((f2 & 0x3FF) << 10) | ((dt & 0xFFF) << 20)



In [9]:
def get_target_zone_points(anchor, point_list,
                           time_min_offset, time_max_offset,
                           freq_min_offset, freq_max_offset):
    """
    Finds points in 'point_list' that lie within time [t1 + time_min_offset, t1 + time_max_offset]
    and frequency [f1 + freq_min_offset, f1 + freq_max_offset].
    
    anchor: (f1, t1) or (t1, f1) – whichever convention you are using.
    point_list: list of (f, t) or (t, f) – must be sorted by the time coordinate if we want to break early.
    time_min_offset, time_max_offset: how far in time we look relative to anchor's time t1.
    freq_min_offset, freq_max_offset: how far in frequency we look relative to anchor's freq f1.
    """
    f1, t1 = anchor
    
    # Time bounds
    t_min = t1 + time_min_offset
    t_max = t1 + time_max_offset
    
    # Frequency bounds
    f_min = f1 + freq_min_offset
    f_max = f1 + freq_max_offset
    
    target_zone_points = []
    for (f2, t2) in point_list:
        # If the list is sorted by time and t2 > t_max, we can break early.
        if t2 > t_max:
            break
        
        if t_min < t2 <= t_max and f_min <= f2 <= f_max:
            target_zone_points.append((f2, t2))
    
    return target_zone_points


In [10]:
import re
import random

def extract_numeric_id(filename):
    """
    Example: 
      - 'queries/1269810_original.mp3' => '1269810'
      - 'tracks/1269810.mp3'           => '1269810'
    Adjust to your naming conventions as needed.
    """
    match = re.search(r'(\d+)', filename)
    return match.group(1) if match else None

In [11]:
def find_best_match_for_query(
    query_name,
    cmap,
    database,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    Given a query's spectrogram cmap (2D Boolean array),
    find the best matching track in the database of fingerprints.

    Returns:
        best_track: str or None
        best_delta: int or None
        best_count: int
        point_list: list of non-zero (freq,time) points in the query
    """
    # Convert the boolean array to a list of points (freq, time)
    point_list = sorted(np.argwhere(cmap).tolist(), key=lambda x: x[1])

    # Dictionary of track_name -> (offset -> count)
    matches = defaultdict(lambda: defaultdict(int))
    
    # For each point in the query, find matching points in the database
    for anchor in point_list:
        target_points = get_target_zone_points(
            anchor,
            point_list,
            time_min_offset=time_min_offset,
            time_max_offset=time_max_offset,
            freq_min_offset=freq_min_offset,
            freq_max_offset=freq_max_offset
        )

        for target_point in target_points:
            h = compute_hash(anchor, target_point)

            # If our hash is found in the database
            if h in database:
                # database[h] = list of (track_name, track_offset)
                for track_name, track_offset in database[h]:
                    delta_offset = track_offset - anchor[1]
                    matches[track_name][delta_offset] += 1
    
    # Find the best match with the highest count
    best_track = None
    best_delta = None
    best_count = 0
    
    for track_name, offset_counts in matches.items():
        for delta_offset, count in offset_counts.items():
            if count > best_count:
                best_count = count
                best_track = track_name
                best_delta = delta_offset

    return best_track, best_delta, best_count, point_list

In [12]:
def load_all_chunks_from_pickle(filename):
    """Load all chunks from a single pickle file into memory."""
    chunks = []
    with open(filename, "rb") as f:
        while True:
            try:
                chunks.append(pickle.load(f))  # Load each chunk and append to the list
            except EOFError:
                break  # Stop when end of file is reached
    return chunks

In [13]:
def match_queries_with_chunks(
    directory,
    database_file,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    Match queries against a database loaded incrementally in chunks.

    Args:
        directory (str): Path to the query files.
        database_file (str): Path to the serialized database file.
        time_min_offset, time_max_offset, freq_min_offset, freq_max_offset (int): 
            Bounds for matching.

    Returns:
        results (list): List of query match results.
    """
    queries = load_filenames(directory)  # Load all query filenames
    results = []
    num_correct = 0
    total_queries = len(queries)
    
    cmaps = {}
    
    for query_name in queries:
        _, cmap = compute_constellation_map_single((query_name, 11, 3))
        cmaps[query_name]=cmap

    bestmatches = {}
    for database_chunk in load_partial_database(database_file):
        print("chunk")
        for query_name in queries:
            track, delta, count, points = find_best_match_for_query(
                query_name,
                cmaps[query_name],
                database_chunk,
                time_min_offset=time_min_offset,
                time_max_offset=time_max_offset,
                freq_min_offset=freq_min_offset,
                freq_max_offset=freq_max_offset
            )
            if query_name in bestmatches: 
                if bestmatches[query_name][2] < count:
                    bestmatches[query_name] = track, delta, count, points
            else: 
                bestmatches[query_name] = track, delta, count, points



    # Print final results
    count3 = 0
    for query_name in queries:
        if int(extract_numeric_id(query_name)) == bestmatches[query_name][0]:
            count3 += 1
        print(query_name, bestmatches[query_name][0])
    print(count3)

    return bestmatches


In [14]:
def load_partial_database(filename):
    """Load chunks from a pickle file incrementally."""
    with open(filename, "rb") as f:
        while True:
            try:
                yield pickle.load(f)  # Yield one chunk at a time
            except EOFError:
                break  # Stop when end of file is reached

In [15]:

def match_queries(
    directory,
    database,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    - Matches each query in cmaps_Q to the best track in the database.
    :param cmaps_Q: dict {query_name: 2D numpy array (spectrogram boolean map)}
    :param database: dict {hash: [(track_name, track_offset), ...]}
    """

    queries = load_filenames(directory) # load all track filenames
        
    results = []
    num_correct = 0
    
    # Collect queries in a list to iterate consistently
    total_queries = len(queries)

    

    for query_name in queries:
        starttime = time.time()
        
        _, cmap = compute_constellation_map_single((query_name, 11, 3))       

        best_track = None
        best_delta = None
        best_count = 0
        point_list = []
        
        for database_chunk in load_partial_database(database_file):
            track, delta, count, points = find_best_match_for_query(
                query_name,
                cmap,
                database_chunk,
                time_min_offset=time_min_offset,
                time_max_offset=time_max_offset,
                freq_min_offset=freq_min_offset,
                freq_max_offset=freq_max_offset
            )

            # Update the best match if this chunk has a better match
            if count > best_count:
                best_track, best_delta, best_count, point_list = track, delta, count, points


        endtime = time.time()
        

        query_id = extract_numeric_id(query_name)
        track_id = extract_numeric_id(best_track) if best_track else None
        correct = (query_id == track_id)
        if correct:
            num_correct += 1

        # Store the result
        results.append((query_name, best_track, best_delta, best_count, correct, (endtime-starttime)*1000))

    # ----------------------------------------------------------------------
    # Print final results
    # ----------------------------------------------------------------------
    accuracy = num_correct / total_queries if total_queries else 0
    print("\nMATCHING RESULTS:")
    sum_duration = 0
    for qname, tname, offset, count, is_correct, duration in results:
        print(f"Query: {qname} => Best match: {tname}, Time: {duration} ms, Offset: {offset}, Count: {count}, Correct: {is_correct}")
        sum_duration += duration

    print(f"\nCorrect matches: {num_correct}/{total_queries}")
    print(f"Average query time: {sum_duration/len(results)} ms")
    print(f"Accuracy: {accuracy*100:.2f}%")

    return results

In [16]:
def save_partial_database(database, filename):
    """Append partial data to a pickle file."""
    with open(filename, "ab") as f:  # Append mode
        pickle.dump(database, f)


# Task 1

## Define configurations

In [17]:
config_1 = {
    "time_min_offset": 0,
    "time_max_offset": 50,
    "freq_min_offset": -10,
    "freq_max_offset": 30
}

config_2 = {
    "time_min_offset": 0,
    "time_max_offset": 30,
    "freq_min_offset": 0,
    "freq_max_offset": 25
}

config_3 = {
    "time_min_offset": 10,
    "time_max_offset": 40,
    "freq_min_offset": -20,
    "freq_max_offset": 20
}

config_4 = {
    "time_min_offset": 0,
    "time_max_offset": 50,
    "freq_min_offset": -10,
    "freq_max_offset": 30
}



## Build the database

In [18]:
s = time.time()
database_1 = build_database("tracks", **config_1)
e = time.time()
print(e-s)

Processed 1000 tracks
Processed 2000 tracks
Processed 3000 tracks
Processed 4000 tracks
Processed 5000 tracks
Processed 6000 tracks
Processed 7000 tracks
Processed 8000 tracks
Processed 9000 tracks
Processed 10000 tracks
Processed 11000 tracks
Processed 12000 tracks
Processed 13000 tracks
Processed 14000 tracks
Processed 15000 tracks
Processed 16000 tracks
Processed 17000 tracks
Processed 18000 tracks
Processed 19000 tracks
Processed 20000 tracks
Processed 21000 tracks
Processed 22000 tracks
Processed 23000 tracks
Processed 24000 tracks
Processed 25000 tracks
Processed 26000 tracks
Processed 27000 tracks
Processed 28000 tracks
Processed 29000 tracks
Processed 30001 tracks
Processed 31000 tracks
Processed 32000 tracks
Processed 33000 tracks
2732.0914878845215


In [ ]:
database_2 = build_database("tracks", **config_2)

In [20]:
s = time.time()

database_3 = build_database("tracks", **config_3)
e = time.time()
print(e-s)

Processed 1000 tracks
Processed 2000 tracks
Processed 3000 tracks
Processed 4000 tracks
Processed 5000 tracks
Processed 6000 tracks
Processed 7000 tracks
Processed 8000 tracks
Processed 9000 tracks
Processed 10000 tracks
Processed 11000 tracks
Processed 12000 tracks
Processed 13000 tracks
Processed 14000 tracks
Processed 15000 tracks
Processed 16000 tracks
Processed 17000 tracks
Processed 18000 tracks
Processed 19000 tracks
Processed 20000 tracks
Processed 21000 tracks
Processed 22000 tracks
Processed 23000 tracks
Processed 24000 tracks
Processed 25000 tracks
Processed 26000 tracks
Processed 27000 tracks
Processed 28000 tracks
Processed 29000 tracks
Processed 30000 tracks
Processed 31000 tracks
Processed 32000 tracks
Processed 33000 tracks
Processed 34000 tracks
2110.072087287903


In [ ]:
database_4 = build_database("tracks", **config_4)

### Store the databases on the harddrive (skip if not needed)

In [ ]:
import pickle 

try:
    database_1
    with open("database_1.pkl", "wb") as file:
        pickle.dump(database_1, file)
except NameError:
    print("database_1 doesn't exist")

try:
    database_2
    with open("database_2.pkl", "wb") as file:
        pickle.dump(database_2, file)
except NameError:
    print("database_2 doesn't exist")

try:
    database_2
    with open("database_3.pkl", "wb") as file:
        pickle.dump(database_2, file)
except NameError:
    print("database_3 doesn't exist")

try:
    database_4
    with open("database_4.pkl", "wb") as file:
        pickle.dump(database_4, file)
except NameError:
    print("database_4 doesn't exist")

# Task 2: Audio Identification

### Load the Databases into memory (skip if not needed)

In [15]:
try:
    with open("database_1.pkl", "rb") as file:
        database_1 = pickle.load(file)
except FileNotFoundError:
    print("database_1 doesn't exist")

try:
    with open("database_2.pkl", "rb") as file:
        database_2 = pickle.load(file)
except FileNotFoundError:
    print("database_2 doesn't exist")

try:
    with open("database_3.pkl", "rb") as file:
        database_3 = pickle.load(file)
except FileNotFoundError:
    print("database_3 doesn't exist")

try:
    with open("database_4.pkl", "rb") as file:
        database_4 = pickle.load(file)
except FileNotFoundError:
    print("database_4 doesn't exist")

database_2 doesn't exist
database_3 doesn't exist
database_4 doesn't exist


### Match all test queries

In [20]:
s = time.time()
matches_1 = match_queries_with_chunks("queries", "partial_database2.pkl", **config_1)
e = time.time()
print(e-s)

chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
queries/1192210_noise.mp3 1192210
queries/91810_coding.mp3 91810
queries/1084710_original.mp3 1084710
queries/242610_mobile.mp3 900733
queries/53310_original.mp3 53310
queries/1192210_mobile.mp3 1192210
queries/1147910_mobile.mp3 1362117
queries/242610_noise.mp3 242610
queries/152310_mobile.mp3 152310
queries/1227910_original.mp3 1227910
queries/963810_noise.mp3 963810
queries/887210_noise.mp3 887210
queries/91810_noise.mp3 91810
queries/1084710_coding.mp3 1084710
queries/1269810_mobile.mp3 1269810
queries/1390710_mobile.mp3 1390710
queries/1400510_noise.mp3 153617
queries/91810_original.mp3 91810
queries/1084710_noise.mp3 1084710
queries/1249910_original.mp3 1249910
queries/1192210_coding.mp3 1192210
queries/262010_coding.mp3 262010
queries/262010_original.mp3 262010
queries/1390710_noise.m

In [ ]:
matches_2 = match_queries("queries", database_2, **config_2)

In [18]:
matches_3 = match_queries_with_chunks("queries", "partial_database2.pkl", **config_3)

chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
chunk
queries/1192210_noise.mp3 1192210
queries/91810_coding.mp3 91810
queries/1084710_original.mp3 1084710
queries/242610_mobile.mp3 1352901
queries/53310_original.mp3 53310
queries/1192210_mobile.mp3 1192210
queries/1147910_mobile.mp3 1066916
queries/242610_noise.mp3 242610
queries/152310_mobile.mp3 152310
queries/1227910_original.mp3 1227910
queries/963810_noise.mp3 963810
queries/887210_noise.mp3 887210
queries/91810_noise.mp3 91810
queries/1084710_coding.mp3 1084710
queries/1269810_mobile.mp3 1269810
queries/1390710_mobile.mp3 1390710
queries/1400510_noise.mp3 1146535
queries/91810_original.mp3 91810
queries/1084710_noise.mp3 1084710
queries/1249910_original.mp3 1249910
queries/1192210_coding.mp3 1192210
queries/262010_coding.mp3 262010
queries/262010_original.mp3 262010
queries/1390710_noise

In [ ]:
matches_4 = match_queries("queries", database_4, **config_4)

# Task 3: Scale up

# Task 4: Report